In [35]:
from torchvision.ops import box_iou
import os
import pandas as pd
import json
import torch
import numpy as np
from collections import defaultdict

In [2]:
BASE_DIR = "/mnt/Enterprise/safal/AI_assisted_microscopy_system/"

In [3]:
gt_annotation_file = os.path.join(
    BASE_DIR,
    "cysts_dataset_all/smartphone_sample/fold_5/smartphone_sample_coco_annos_val.json",
)

pred_annotation_file = os.path.join(
    BASE_DIR,
    "outputs/smartphone_sample/faster_rcnn_x101_32x8d_fpn_mstrain_3x_coco/fold_5/results.bbox.json"
)

In [4]:
gt_annotations = json.load(open(gt_annotation_file))
pred_annotations = json.load(open(pred_annotation_file))

In [5]:
gt_annotations_df = pd.DataFrame(gt_annotations["annotations"])
pred_annotations_df = pd.DataFrame(pred_annotations)

In [8]:
# change bbox width and height to x2, y2
pred_annotations_df["bbox"] = pred_annotations_df["bbox"].apply(
    lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
)
gt_annotations_df["bbox"] = gt_annotations_df["bbox"].apply(
    lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
)

In [74]:
# get the image ids which have both category ids 0 and 1
image_ids = gt_annotations_df["image_id"].unique()
image_ids = [
    image_id
    for image_id in image_ids
    if 0 in gt_annotations_df[gt_annotations_df["image_id"] == image_id]["category_id"].values
    and 1 in gt_annotations_df[gt_annotations_df["image_id"] == image_id]["category_id"].values
]

image_ids


[240, 259, 405, 505, 744]

In [110]:
gt_annotations_df.image_id.unique()

array([   1,    6,   24,   26,   36,   37,   48,   53,   56,   57,   81,
         94,   99,  107,  123,  136,  141,  146,  151,  154,  158,  159,
        181,  198,  203,  208,  240,  243,  244,  245,  255,  259,  282,
        284,  287,  305,  309,  310,  320,  325,  326,  358,  359,  360,
        362,  372,  373,  397,  398,  402,  405,  422,  432,  448,  456,
        459,  486,  496,  498,  503,  505,  530,  539,  547,  550,  557,
        560,  564,  566,  568,  574,  579,  588,  600,  608,  654,  655,
        660,  661,  663,  669,  681,  682,  683,  693,  702,  704,  705,
        713,  718,  725,  727,  744,  749,  772,  779,  792,  794,  795,
        799,  803,  804,  805,  807,  816,  817,  825,  845,  855,  869,
        876,  895,  896,  897,  905,  920,  931,  934,  946,  948,  954,
        976,  987,  993, 1028, 1032, 1044, 1048])

In [61]:
pred_annotations_df[pred_annotations_df.image_id == 37]

,image_id,bbox,score,category_id
17,37,"[1719.8165283203125, 1541.7166748046875, 1764....",0.526125,0
18,37,"[1716.9451904296875, 1537.9757080078125, 1770....",0.065791,1


In [21]:
# get the categories
categories = sorted(gt_annotations_df.category_id.unique())
categories


[0, 1]

In [119]:
# Precision x Recall is obtained individually by each class
# Loop through each class and calculate the precision and recall

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)

for category in categories:
    # get the ground truth annotations for the current class
    gt_annotations_df_class = gt_annotations_df[
        gt_annotations_df.category_id == category
    ]
    # get the predicted annotations for the current class
    pred_annotations_df_class = pred_annotations_df[
        pred_annotations_df.category_id == category
    ]

    # sort the predicted annotations by score
    pred_annotations_df_class = pred_annotations_df_class.sort_values(
        by="score", ascending=False
    )

    # filter predictions with score > 0.3
    pred_annotations_df_class = pred_annotations_df_class[
        pred_annotations_df_class.score > 0.3
    ]

    true_positives_class = 0
    false_positives_class = 0

    # get image ids for the current class from both ground truth and predicted annotations
    image_ids = pred_annotations_df_class["image_id"].unique()
    images_len = len(image_ids)

    for image in image_ids:
        # get the ground truth annotations for the current image
        gt_annotations_df_image = gt_annotations_df_class[
            gt_annotations_df_class.image_id == image
        ]
        # get the predicted annotations for the current image
        pred_annotations_df_image = pred_annotations_df_class[
            pred_annotations_df_class.image_id == image
        ]

        # get the ground truth bounding boxes
        gt_bboxes = list(gt_annotations_df_image.bbox.values)
        gt_bboxes = torch.tensor(gt_bboxes)

        # get the predicted bounding boxes

        # only take the predicted bounding boxes which have a score > 0.3
        pred_bboxes = list(pred_annotations_df_image.bbox.values)
        pred_bboxes = torch.tensor(pred_bboxes)


        if len(gt_bboxes) == 0:
            false_positives_class += len(pred_bboxes)
            continue

        # get the intersection over union for each predicted bounding box
        ious = box_iou(gt_bboxes, pred_bboxes)

        # get the maximum iou for each ground truth bounding box
        max_ious, _ = torch.max(ious, dim=0)

        # get the indices of the predicted bounding boxes with iou > 0.5
        tp_indices = torch.where(max_ious >= 0.5)[0]
        # print(ious)

        # get the indices of the predicted bounding boxes with iou < 0.5
        fp_indices = torch.where(max_ious < 0.5)[0]

        # update the true positives and false positives
        true_positives_class += len(tp_indices)
        false_positives_class += len(fp_indices)

    # print actual number of ground truth annotations and predicted annotations for the current class
    print(
        "Actual:",
        gt_annotations_df_class.shape[0],
        "Predicted:",
        pred_annotations_df_class.shape[0],
    )

    # print true positives and false positives for the current class
    print(
        "True positives:",
        true_positives_class,
        "False positives:",
        false_positives_class,
    )
    # calculate the precision and recall
    precision = true_positives_class / (true_positives_class + false_positives_class)
    recall = true_positives_class / gt_annotations_df_class.shape[0]

    category_name = gt_annotations["categories"][category]["name"]
    print(f"Category: {category_name}, Precision: {precision}, Recall: {recall}")


Actual: 110 Predicted: 163
True positives: 86 False positives: 77
Category: Crypto, Precision: 0.5276073619631901, Recall: 0.7818181818181819
Actual: 59 Predicted: 82
True positives: 50 False positives: 32
Category: Giardia, Precision: 0.6097560975609756, Recall: 0.847457627118644


In [26]:
# get bbox for image_id=1
gt_bbox_1 = gt_annotations_df[gt_annotations_df["image_id"] == 259]

In [27]:
pred_bbox_1 = pred_annotations_df[pred_annotations_df["image_id"] == 259]

In [28]:
gt_bbox_1

,image_id,id,category_id,bbox,segmentation,iscrowd,area
43,259,171,1,"[1091.0, 1389.0, 1177.0, 1497.0]",[],0,9288.0
44,259,172,0,"[878.0, 2338.0, 944.0, 2414.0]",[],0,5016.0


In [29]:
pred_bbox_1

,image_id,bbox,score,category_id
86,259,"[1105.5439453125, 1390.0980224609375, 1171.617...",0.998726,1


In [20]:
pred_bbox_tensor = torch.tensor(pred_bbox_1["bbox"].values.tolist())
pred_bbox_tensor

tensor([[1677.4498, 1608.5356, 1737.7509, 1665.9437],
        [1373.4507, 1647.2849, 1428.7148, 1704.8878],
        [1364.7380, 1639.0117, 1435.4178, 1721.2365],
        [1674.5179, 1607.7670, 1755.6367, 1674.7997]])

In [21]:
gt_bbox_tensor = torch.tensor(gt_bbox_1["bbox"].values.tolist())
gt_bbox_tensor

tensor([[1367., 1633., 1453., 1709.],
        [1643., 1611., 1741., 1687.]])

In [23]:
pairwise_iou = box_iou(gt_bbox_tensor, pred_bbox_tensor)

In [24]:
pairwise_iou

tensor([[0.0000, 0.4871, 0.6335, 0.0000],
        [0.4361, 0.0000, 0.0000, 0.4907]])

In [38]:
for i in pairwise_iou.numpy():
    print(i.argmax())

2
3


In [44]:
from mmdet.core.evaluation.bbox_overlaps import bbox_overlaps

bbox_overlaps(gt_bbox_tensor.numpy(), pred_bbox_tensor.numpy())

array([[0.        , 0.48705268, 0.63346106, 0.        ],
       [0.43613747, 0.        , 0.        , 0.49068674]], dtype=float32)